In [9]:
from Bio import SeqIO, AlignIO
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
from os import listdir


# wuhan_file = "wuhan.fasta"
beta_file = "prueba_3.fasta"
beta_file = "lineage_B_1_351.fasta"

secuencias_wuhan = next(SeqIO.parse(wuhan.fasta, "fasta"))
secuencias_beta = SeqIO.parse(beta_file, "fasta")


In [2]:
for i,sec in tqdm(enumerate(secuencias_beta)):
    file =open(f"./secuencias/{i}.fasta", "w")
    file.write(">" + secuencias_wuhan.id + "\n" + str(secuencias_wuhan.seq) + "\n")
    file.write(">" + sec.description + "\n" + str(sec.seq))
    file.close()
   

39302it [00:08, 4442.73it/s]


In [10]:
files = listdir("./secuencias/")
directorio = '/home/jorge/Documents/script_covid'

for i in tqdm(files, total=39302):
    name = i.replace(".fasta", ".aln")
    comando = f'/usr/bin/mafft --thread 16 --retree 1 --clustalout --inputorder ./secuencias/{i} > ./alineadas/{name}'
    subprocess.run(comando, shell=True, cwd=directorio, stdout = subprocess.DEVNULL, stderr = subprocess.DEVNULL)


In [11]:
def comparar(file, pos_sin_gap_sec1):
    aln = AlignIO.read(file, "clustal")
    sec1 = str(aln[0].seq)
    sec2 = str(aln[1].seq)
    pos_con_gap_sec1 = -1
    gaps_sec1 = 0
    cont = 0

    while cont <= pos_sin_gap_sec1:
        pos_con_gap_sec1 += 1
        if sec1[pos_con_gap_sec1] != '-':
            cont += 1
    
    nucleotido_sec1 = sec1[pos_con_gap_sec1-1]
    nucleotido_sec2 = sec2[pos_con_gap_sec1-1]

    pos_sin_gap_sec2 = pos_con_gap_sec1 - sec2[0:pos_con_gap_sec1].count('-')
    
    return [pos_con_gap_sec1, pos_sin_gap_sec1, nucleotido_sec1, pos_sin_gap_sec2, nucleotido_sec2]


In [12]:
def leer_id(file):
    file = file.replace(".aln", ".fasta")
    id_name = [i for i in SeqIO.parse('./secuencias/'+file, "fasta")][-1].id
    # if id_name is None:
    #     id_name = f'SN-{file}'
    return id_name


In [13]:
dir_aln = "./alineadas/"
lista_aln = listdir(dir_aln)

In [14]:
def resultados(pos, esperado):
    bases = ['a', 't', 'g', 'c']
    bases.remove(esperado)
    resultados_dict = {i:[leer_id(i)]+comparar(dir_aln+i, pos) for i in tqdm(lista_aln)}
    df = pd.DataFrame.from_dict(resultados_dict, orient='index', columns=['descripcion', 'pos_con_gap_sec1', 'pos_sin_gap_sec1', 'nucleotido_sec1', 'pos_sin_gap_sec2', 'nucleotido_sec2'])
    df['resultado'] = np.where(df['nucleotido_sec2'] == esperado, 1, np.where(np.isin(df['nucleotido_sec2'], bases), 0, np.nan))
    df.to_csv(f"resultados-{pos}.csv")
    return df

In [15]:
df1 = resultados(21801, 'c')
df2 = resultados(23012, 'a')



  0%|          | 0/39302 [00:00<?, ?it/s]

100%|██████████| 39302/39302 [01:42<00:00, 383.99it/s]


In [16]:
df_final = df1.merge(df2, left_index=True, right_index=True, suffixes=('_21801', '_23012'))
df_final['resultado'] = df_final['resultado_21801'] * df_final['resultado_23012']

In [17]:
df_final.to_csv("resultados-final.csv")
df_final

,descripcion_21801,pos_con_gap_sec1_21801,pos_sin_gap_sec1_21801,nucleotido_sec1_21801,pos_sin_gap_sec2_21801,nucleotido_sec2_21801,resultado_21801,descripcion_23012,pos_con_gap_sec1_23012,pos_sin_gap_sec1_23012,nucleotido_sec1_23012,pos_sin_gap_sec2_23012,nucleotido_sec2_23012,resultado_23012,resultado
6208.aln,hCoV-19/Botswana/R16B85_BHP_AAB28492/2021|2021...,21917,21801,a,21747,n,NaN,hCoV-19/Botswana/R16B85_BHP_AAB28492/2021|2021...,23128,23012,g,22958,n,NaN,NaN
32100.aln,hCoV-19/Reunion/HCL021128408801/2021|2021-07-0...,21801,21801,a,21783,c,1.0,hCoV-19/Reunion/HCL021128408801/2021|2021-07-0...,23012,23012,g,22985,a,1.0,1.0
20988.aln,hCoV-19/Philippines/PH-PGC-58213/2021|2021-07-...,21801,21801,a,21762,c,1.0,hCoV-19/Philippines/PH-PGC-58213/2021|2021-07-...,23012,23012,g,22964,a,1.0,1.0
27007.aln,hCoV-19/Finland/4666/2021|2021-03-09|2021-05-03,21801,21801,a,21767,c,1.0,hCoV-19/Finland/4666/2021|2021-03-09|2021-05-03,23012,23012,g,22969,a,1.0,1.0
38503.aln,hCoV-19/Kenya/SS7150/2021|2021-04-07|2022-04-07,21801,21801,a,21742,c,1.0,hCoV-19/Kenya/SS7150/2021|2021-04-07|2022-04-07,23012,23012,g,22944,a,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38763.aln,hCoV-19/France/HDF-P213-21164M0967/2021|2021-0...,21801,21801,a,21762,c,1.0,hCoV-19/France/HDF-P213-21164M0967/2021|2021-0...,23012,23012,g,22964,a,1.0,1.0
33932.aln,hCoV-19/USA/NC-CDC-STM-000045790/2021|2021-03-...,21801,21801,a,21762,c,1.0,hCoV-19/USA/NC-CDC-STM-000045790/2021|2021-03-...,23012,23012,g,22964,a,1.0,1.0
21252.aln,hCoV-19/Germany/BY-RKI-I-044169/2021|2021-03-1...,21801,21801,a,21767,c,1.0,hCoV-19/Germany/BY-RKI-I-044169/2021|2021-03-1...,23012,23012,g,22969,a,1.0,1.0
34575.aln,hCoV-19/Sweden/3945000664/2021|2021-03-26|2021...,21810,21801,a,21792,c,1.0,hCoV-19/Sweden/3945000664/2021|2021-03-26|2021...,23021,23012,g,22994,a,1.0,1.0


In [18]:
df_final['descripcion_21801'].is_unique

False